### קוד מבוא

In [15]:
import os
import pathlib
import pandas as pd
import geopandas as gpd
import fiona

In [16]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [17]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [18]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [19]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [20]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [21]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [22]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

In [23]:
def find_files_with_pattern(folder_path, pattern):
    """
    Find files in a directory that match a certain pattern.
    
    Args:
    - directory (str): The directory path.
    - pattern (str): The pattern to search for in file names.
    
    Returns:
    - List of file paths matching the pattern.
    """
    files = []
    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if pattern in filename:
                files.append(os.path.join(root, filename))
    return files

In [24]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

## קוד

In [25]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [26]:
year=['2025','2030','2035','2040','2045','2050']

In [27]:
matching_files=find_files_with_pattern(r'{}\Intermediates'.format(software_data_folder_location), '_pop_2040_bau')
file = pathlib.Path(matching_files[0])
v_date = file.name[:6]
folder = str(file.parent)
v_date

'240318'

In [28]:
for y in year:
        save_taz_path=r'{}\Intermediates'.format(software_data_folder_location)

        pop_to_change=up_load_df(save_taz_path,'{}_pop_{}_bau'.format(v_date,y))

        pop_goals=up_load_df(save_taz_path,'gov_goals_{}_BAU'.format(y))

        shp=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

        folder_path=r'{}\background_files'.format(software_data_folder_location)

        taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

        lst_change_sector=list(taz_sector_change.Taz_num)

        shp.loc[(shp['Taz_num'].isin(lst_change_sector)),'main_secto']='U_Orthodox'

        col=['Taz_num','main_secto','zonetype']

        pop_to_change=pop_to_change.merge(shp[col],on='Taz_num',how='left')
        
        pop_Palestinian_to_add=pop_to_change.loc[pop_to_change['main_secto']=="Palestinian"]
        
        pop_to_change=pop_to_change.loc[~(pop_to_change['main_secto']=="Palestinian")]
        
        pop_to_change['group']= pop_to_change['main_secto']

        pop_to_change.loc[(pop_to_change['main_secto']=="Arab" )|(pop_to_change['main_secto']=="arabs_behined_seperation_wall" ),'group']= 'arab'

        age_group=[ 'pop_0',
         'pop_10',
         'pop_15',
         'pop_20',
         'pop_25',
         'pop_30',
         'pop_35',
         'pop_40',
         'pop_45',
         'pop_5',
         'pop_50',
         'pop_55',
         'pop_60',
         'pop_65',
         'pop_70',
         'pop_75up']

        pop_goals.loc[pop_goals['district']=="Jerusalem District",'zonetype']='Jerusalem'

        pop_goals.loc[pop_goals['district']=="Yehuda and Shomron",'zonetype']='Judea and Samaria'

        pop_goals.loc[pop_goals['group']=="jew_not_haredi",'group']='Jewish'

        pop_goals.loc[pop_goals['group']=="haredi",'group']='U_Orthodox'

        pop_goals['age_group']='pop_'+pop_goals['age'].astype(str)

        pop_goals.loc[pop_goals['age']>74,'age_group']='pop_75up'

        pop_goals=pop_goals.pivot_table(index=['zonetype', 'group', 'age_group'],aggfunc=sum)['pop'].reset_index()

        zonetype=['Jerusalem','Judea and Samaria']

        group=['Jewish','U_Orthodox','arab']

        for z in zonetype:
            for g in group:
                if not((z=="Judea and Samaria") & (g=="arab")):
                    for a in age_group:
                        pop_to_change.loc[(pop_to_change['group']==g)&(pop_to_change['zonetype']==z),'sum_{}'.format(a)]=pop_to_change.loc[(pop_to_change['group']==g)&(pop_to_change['zonetype']==z)]['{}'.format(a)].sum()
                        pop_to_change.loc[(pop_to_change['group']==g)&(pop_to_change['zonetype']==z),'pre_{}'.format(a)]=pop_to_change['{}'.format(a)]/pop_to_change['sum_{}'.format(a)]
                        total_gov_age_num=pop_goals.loc[(pop_goals['zonetype']==z) & (pop_goals['group']==g) & (pop_goals['age_group']==a)]['pop'].item()
                        pop_to_change.loc[(pop_to_change['group']==g)&(pop_to_change['zonetype']==z),'{}'.format(a)]=pop_to_change['pre_{}'.format(a)]*total_gov_age_num

        pop_to_change['pop_old']=pop_to_change['pop']

        col=['pop_0',
         'pop_10',
         'pop_15',
         'pop_20',
         'pop_25',
         'pop_30',
         'pop_35',
         'pop_40',
         'pop_45',
         'pop_5',
         'pop_50',
         'pop_55',
         'pop_60',
         'pop_65',
         'pop_70',
         'pop_75up']

        pop_to_change['pop']=pop_to_change[col].sum(axis=1)

        pop_to_change['aprt_old']=pop_to_change['aprt']

        pop_to_change['aprt']=pop_to_change['aprt']*(pop_to_change['pop']/pop_to_change['pop_old'])
        
        
        pop_to_change=pd.concat([pop_to_change,pop_Palestinian_to_add],axis=0)
        
        save_taz_path=r'{}\Intermediates\change_pop'.format(software_data_folder_location)

        pop_to_change.to_excel('{}\{}_pop_{}_bau_changed.xlsx'.format(save_taz_path,file_date,y))

C:\Users\dpere\AppData\Local\Temp\ipykernel_11384\870159495.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Jerusalem' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pop_goals.loc[pop_goals['district']=="Jerusalem District",'zonetype']='Jerusalem'
C:\Users\dpere\AppData\Local\Temp\ipykernel_11384\870159495.py:59: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pop_goals=pop_goals.pivot_table(index=['zonetype', 'group', 'age_group'],aggfunc=sum)['pop'].reset_index()
C:\Users\dpere\AppData\Local\Temp\ipykernel_11384\870159495.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Jerusalem' has dtype incompatible wi